# Find cluster related terms using LIME

## Load the data

In [1]:
root_dir = '../..'
data_dir = 'data'
corpus_dir = 'corpus'
src_dir = 'src'

In [2]:
import os 
import sys

In [3]:
sys.path.append(os.path.join(root_dir, src_dir))

In [4]:
dataset_name = 'alaska'
test_name = 'sampling_terms_test2'

In [5]:
corpus_filename = f'{dataset_name}_corpus.json'
corpus_filepath = os.path.join(root_dir, data_dir, corpus_dir, corpus_filename)

In [6]:
chunks_filename = f'{dataset_name}_chunks.json'
chunks_filepath = os.path.join(root_dir, data_dir, corpus_dir, chunks_filename)

In [7]:
from training import TrainingCorpus

In [8]:
corpus = TrainingCorpus()
corpus.load(corpus_filepath)
corpus.load_chunks(chunks_filepath)

---

## Load the model

Check if GPU is available

In [9]:
import torch
print(torch.cuda.is_available())

True


In [10]:
from model import BertModel

Using TensorFlow backend.


In [11]:
model_dir = f'models/{dataset_name}_bert'
model_dir_path = os.path.join(root_dir, data_dir, model_dir)

In [12]:
model = BertModel(model_dir_path, batch_size=512, use_cuda=True, from_tf=True)

All TF 2.0 model weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


---

## Find relevant terms for each cluster label using LIME

### Instantiate TermFinder

In [13]:
from termfinder import LimeTermFinder

In [14]:
term_finder = LimeTermFinder(model, corpus, min_fts=5, max_fts=10)

### Retrieve predicted labels for each instance in the corpus

In [15]:
import numpy as np

In [16]:
label_to_data_idx_dict = model.label_to_data_idx(corpus)

In [17]:
len(label_to_data_idx_dict)

20

### Retrieve relevant terms using LimeTermFinder

In [18]:
from tqdm.notebook import tqdm

In [19]:
df_data = []

In [20]:
import time
import datetime

start_time = time.time()

for label_idx, data_idxs in tqdm(label_to_data_idx_dict.items()):
    for data_idx in tqdm(data_idxs, desc=f'Relevant terms for entity {label_idx}', leave=False):
        relevant_terms = term_finder.get_relevant_terms(data_idx, label_idx)

        if relevant_terms:
            for term, weight in relevant_terms.items():
                dict_entry = {'label': corpus.labels[label_idx], 
                              'term': term, 
                              'weight': weight, 
                              'data_id': corpus.docs[data_idx]}
                df_data.append(dict_entry)

end_time = time.time()

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [21]:
delta_t = end_time - start_time
elapsed_time = str(datetime.timedelta(seconds=delta_t))
print(f'Elapsed time: {elapsed_time}')

Elapsed time: 1:42:19.190054


Build a DataFrame out of `df_data`

In [22]:
df_data[:3]

[{'label': 'ENTITY#44',
  'term': 'nikon_d3200_dslr_camera',
  'weight': 0.8654738475187627,
  'data_id': 0},
 {'label': 'ENTITY#44',
  'term': 'nikon_d3200',
  'weight': 0.8614877851152007,
  'data_id': 1},
 {'label': 'ENTITY#44',
  'term': 'nikon_d3200',
  'weight': 0.76109970136487,
  'data_id': 2}]

In [23]:
import pandas as pd

In [24]:
relevant_terms_df = pd.DataFrame(df_data)

In [25]:
relevant_terms_df.head()

,label,term,weight,data_id
0,ENTITY#44,nikon_d3200_dslr_camera,0.865474,0
1,ENTITY#44,nikon_d3200,0.861488,1
2,ENTITY#44,nikon_d3200,0.761100,2
3,ENTITY#44,digital_dslr_camera,0.039113,2
4,ENTITY#44,nikon_d3200_digital_dslr_camera,0.808393,3


---
## Save retrieved terms to a file

In [26]:
terms_dir = 'terms'
filename = f'relevant_terms_{dataset_name}_bert_{test_name}.csv'
filepath = os.path.join(root_dir, data_dir, terms_dir, filename)
filepath

'../../data/terms/relevant_terms_alaska_bert_sampling_terms_test2.csv'

In [27]:
relevant_terms_df.to_csv(filepath, encoding='utf-8', index=False)

---